In [5]:
import os
import requests
import base64
import json
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")

In [9]:
import os
import requests
import base64
import pandas as pd
import re
from dotenv import load_dotenv

# GPT-4o API 엔드포인트
API_URL = "https://api.openai.com/v1/chat/completions"

# 이미지 파일 경로
IMAGE_FOLDER = "image/"


def encode_image(image_path):
    """이미지를 Base64로 인코딩"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def ask_gpt4o_for_product_info(image_path):
    """GPT-4o에게 제품명, 가격, 용량을 추출 요청"""
    base64_image = encode_image(image_path)

    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

    data = {
        "model": "gpt-4o-2024-11-20",
        "temperature": 0.3,
        "messages": [
            {
                "role": "system",
                "content": "Instructions:\n1. Detect the price tags in the image and extract the text from those price tags.\n2. For each price tag (label), identify the following five items and output them in one line separated by commas (,):\n   - Product name\n   - Capacity\n   - Product code (13 digits)\n   - Regular price (in KRW)\n   - Discount price (in KRW)\n3. If the discount price is not provided, use the same value as the regular price.\n4. If any item is incorrect or unclear, please write '식별불가' for that item.\n5. Always add '원' after the price.\n6. Ignore the phrase '마이홈플러스멤버십 제시시 50% 할인', as it is not product information.\n7. If, for any reason, none of the items can be found, write '식별불가' for all items.\n8. Do not arbitrarily add or omit any information from the original text; present it exactly as identified.\n9. Perform all tasks without any refusal to the greatest extent possible.\n10. Only use the information from the price tag.\n\n출력 형식 예시(필수 준수)\n각 상품당 한 줄로: [제품명], [용량], [상품코드(13자리)], [정상가], [할인가]\n예시: 클립램프 영양립밤, 식별불가, 8809704424769, 1,090원, 1,090원",
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "이 이미지에서 가격표를 찾아 한 줄로 반환해줘.",
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            },
        ],
    }

    response = requests.post(API_URL, headers=headers, json=data)

    # 디버깅용 전체 응답 출력
    print("\n📌 GPT-4o API 원본 응답:")
    print(response.json())

    return response.json()


def process_images_from_folder(folder_path):
    """폴더 내 모든 이미지 파일을 처리하고 CSV로 저장"""
    all_product_data = []

    # 폴더 내 이미지 파일 목록 가져오기
    image_files = [
        f
        for f in os.listdir(folder_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]

    for img_file in image_files:
        image_path = os.path.join(folder_path, img_file)
        print(f"\n🚀 처리 중: {image_path}")

        result = ask_gpt4o_for_product_info(image_path)

        # 모델의 응답에서 추출된 텍스트 가져오기
        extracted_text = (
            result.get("choices", [{}])[0].get("message", {}).get("content", "").strip()
        )

        print(f"\n📌 GPT-4o 반환 텍스트 ({img_file}):")
        print(repr(extracted_text))  # 디버깅용 (공백/줄바꿈 문제 확인)

        # 🔹 "\n\n" -> "\n" 로 변환하여 줄바꿈 정리
        extracted_text = extracted_text.replace("\n\n", "\n")

        # 🔹 가격 쉼표(`,`)를 임시로 다른 문자(#)로 변환하여 데이터 분리 문제 방지
        temp_text = re.sub(r"(\d{1,3}),(\d{3}원)", r"\1#\2", extracted_text)

        # 🔹 리스트 변환 시 공백 제거 후 줄바꿈 기준으로 분리
        lines = [line.strip() for line in temp_text.split("\n") if line.strip()]

        for line in lines:
            items = [
                item.strip().replace("#", ",") for item in line.split(",")
            ]  # 쉼표 임시 문자 복원
            print(f"\n📌 개별 라인 분석 ({img_file}): {repr(items)}")  # 디버깅용

            if (
                len(items) == 5
            ):  # 제품명, 용량, 정가, 할인가가 정상적으로 들어갔는지 확인
                all_product_data.append(
                    {
                        "파일명": img_file,
                        "제품명": items[0],
                        "용량": items[1],
                        "상품코드": items[2],
                        "정상가": items[3],
                        "할인가": items[4],
                    }
                )

    # 데이터프레임 생성
    df = pd.DataFrame(all_product_data)

    # CSV 파일 저장
    output_csv = "extracted_prices.csv"
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")

    print(f"\n✅ 가격표 정보가 {output_csv} 파일에 저장되었습니다.")
    print(df)  # 최종 데이터 확인


# 실행
process_images_from_folder(IMAGE_FOLDER)


🚀 처리 중: image/립(1).jpg

📌 GPT-4o API 원본 응답:
{'id': 'chatcmpl-BCGEzljieuCNyEeUg1RwI2WT3mxxp', 'object': 'chat.completion', 'created': 1742262065, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '별의 포켓몬 키링립밤(이브이), 4.5g 1입, 8801051313255, 12,900원, 12,900원', 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1105, 'completion_tokens': 42, 'total_tokens': 1147, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': 'fp_178d5ba2c0'}

📌 GPT-4o 반환 텍스트 (립(1).jpg):
'별의 포켓몬 키링립밤(이브이), 4.5g 1입, 8801051313255, 12,900원, 12,900원'

📌 개별 라인 분석 (립(1).jpg): ['별의 포켓몬 키링립밤(이브이)', '4.5g 1입', '8801051313255', '12,900원', '12,900원']

🚀 처리 중: image/립(10).jpg

📌 GPT-4o API 원본 응답:
{'id': 'chatcmpl-BCGF3c0qgTc

In [10]:
from roboflow import Roboflow

TARGET_DIR = "image/object"


rf = Roboflow(api_key="Da0SeD9LCEl6rsnMvurC")

# 프로젝트 이름 확인 후 변경
project = rf.workspace("n2solution").project("yolo_ocr-wpyuv")
version = project.version(1)

# 데이터셋 다운로드 및 저장
dataset = version.download("yolov5-obb", location=TARGET_DIR)

print(f"📌 데이터셋이 '{TARGET_DIR}' 경로에 저장되었습니다.")

loading Roboflow workspace...
loading Roboflow project...
📌 데이터셋이 'image/object' 경로에 저장되었습니다.


In [ ]:
from ultralytics import YOLO

# YOLO 모델 로드 (기본 YOLOv8 모델)
model = YOLO("yolov8n.pt")

# 데이터셋 경로 설정
data_yaml = "dataset/data.yaml"  # YOLO 학습 데이터 설정 파일

# YOLO 학습 실행
model.train(data=data_yaml, epochs=50, imgsz=640)

가격표 감지된 영역: [(596, 13, 747, 64), (221, 42, 344, 97), (146, 93, 188, 141), (588, 114, 673, 157), (1006, 250, 1020, 264), (604, 442, 663, 476), (145, 519, 179, 531), (551, 550, 650, 582), (492, 582, 560, 603), (872, 623, 877, 628), (124, 600, 216, 626), (851, 631, 868, 651), (875, 620, 908, 663), (915, 620, 955, 663), (965, 646, 993, 656), (935, 667, 938, 681), (947, 672, 960, 680), (584, 1043, 601, 1064), (1033, 1076, 1048, 1083), (646, 1102, 660, 1116), (152, 1155, 154, 1157), (402, 1172, 429, 1192), (436, 1176, 461, 1185), (667, 1185, 696, 1205), (434, 1227, 516, 1251)]


In [42]:
!pip install labelImg


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 25.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------------------- -------------------- 3.4/6.9 MB 18.3 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 18.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
   --- ------------------------------------ 5.0/50.1 MB 27.6 MB/s eta 0:00:02
   ------- -------------------------------- 9.4/50.1 MB 24.5 MB/s eta 0:00:02
   ---------- ----------------------------- 13.6/50.1 MB 22.5 MB/s eta 0:00


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
labelImg

NameError: name 'labelImg' is not defined